<h1 align="center"> Capstone Project - Data </h1>

### Data

1. Toronto City crime data that contains list of crimes committed in various Neighborhoods along with the latitude and longitude of the location of crime and other relevant data. 
    
    Data source : https://opendata.arcgis.com/datasets/98f7dde610b54b9081dfca80be453ac9_0.csv?outSR=%7B%22wkid%22%3A102100%2C%22latestWkid%22%3A3857%7D

    Description : This data set contains the required information for crimes and location of various crimes, we will use this data set to explore crime rates of various neighborhoods.

#### Preview of data

In [1]:
import pandas as pd

crime = pd.read_csv('MCI_2014_to_2018.csv')
crime.head()

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,...,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Lat,Long,ObjectId
0,-79.343094,43.668449,214,GO-20141948968,2014-04-24T11:29:00.000Z,2014-04-24T12:46:00.000Z,Commercial,1610,200,Robbery - Mugging,...,114.0,Thursday,11,Robbery,D55,68,North Riverdale (68),43.668449,-79.343094,1
1,-79.507927,43.759285,215,GO-20141950728,2014-04-24T13:00:00.000Z,2014-04-24T15:58:00.000Z,House,2120,200,B&E,...,114.0,Thursday,13,Break and Enter,D31,24,Black Creek (24),43.759285,-79.507927,2
2,-79.501663,43.697552,216,GO-20141956416,2014-04-25T13:20:00.000Z,2014-04-25T13:52:00.000Z,Apartment,1430,100,Assault,...,115.0,Friday,13,Assault,D12,30,Brookhaven-Amesbury (30),43.697552,-79.501663,3
3,-79.571510,43.721703,217,GO-20141956867,2014-04-24T17:00:00.000Z,2014-04-25T10:30:00.000Z,Outside,1430,100,Assault,...,114.0,Thursday,17,Assault,D23,4,Rexdale-Kipling (4),43.721703,-79.571510,4
4,-79.503487,43.663891,218,GO-20141957505,2014-04-25T16:00:00.000Z,2014-04-25T16:15:00.000Z,Apartment,1430,100,Assault,...,115.0,Friday,16,Assault,D11,114,Lambton Baby Point (114),43.663891,-79.503487,5


In [2]:
crime.drop(['offence','occurrenceday','ObjectId','reportedyear','reportedmonth','reportedday',
            'reporteddayofyear','reporteddayofweek','reportedhour','occurrenceyear','occurrencemonth',
            'Index_','event_unique_id','occurrencedate','reporteddate','premisetype','ucr_code','ucr_ext',
            'occurrencedayofyear','X','Y','occurrencedayofweek','occurrencehour','Division'],
           axis=1, inplace=True)
crime.rename({'MCI':'Offense'},axis=1, inplace=True)

# This data willbe used directly 
crime.head()

,Offense,Hood_ID,Neighbourhood,Lat,Long
0,Robbery,68,North Riverdale (68),43.668449,-79.343094
1,Break and Enter,24,Black Creek (24),43.759285,-79.507927
2,Assault,30,Brookhaven-Amesbury (30),43.697552,-79.501663
3,Assault,4,Rexdale-Kipling (4),43.721703,-79.571510
4,Assault,114,Lambton Baby Point (114),43.663891,-79.503487


2. Venues of various categories in each neighborhood of Toronto city.
        
    Data source : Fousquare API
    
    Description : By using this API we will get all the required venues in each neighborhood such as residence areas, shops, restaurants.etc. We can further filter these venues to find the best amongst them.

#### Preview of API

In [16]:
import json
from pandas.io.json import json_normalize
import requests

VERSION = '20180605' # Foursquare API version
CLIENT_ID = '-' # your Foursquare ID
CLIENT_SECRET = '-' # your Foursquare Secret

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: -
CLIENT_SECRET:-


In [13]:
# type your answer here
LIMIT=100
radius=500
latitude=43.653226
longitude=-79.3831843

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=4FUTEHMFHAVMGT15NO0TCPG1NS3KLN20NOSUMMQI2AILQE5Q&client_secret=YD1SAX0MPSC1KALSLZFOX1PHP4N3EQI0IPWGKUVDIY0Q2UTU&v=20180605&ll=43.653226,-79.3831843&radius=500&limit=100'

In [14]:
results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results["response"]['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Indigo,Bookstore,43.653515,-79.380696
3,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
4,Old City Hall,Monument / Landmark,43.652009,-79.381744
